# Export data to files with Suitcase

In this notebook you will:

* Export saved data to CSV and TIFF.
* See a preview of how exporting data live during a scan works.

## Configuration
Below, we will connect to EPICS IOC(s) controlling simulated hardware in lieu of actual motors, detectors. The IOCs should already be running in the background. Run this command to verify that they are running: it should produce output with RUNNING on each line. In the event of a problem, edit this command to replace `status` with `restart all` and run again.

In [1]:
!supervisorctl -c supervisor/supervisord.conf status

decay                            RUNNING   pid 4899, uptime 0:00:33
mini_beamline                    RUNNING   pid 4900, uptime 0:00:33
random_walk                      RUNNING   pid 4901, uptime 0:00:33
random_walk_horiz                RUNNING   pid 4902, uptime 0:00:33
random_walk_vert                 RUNNING   pid 4903, uptime 0:00:33
simple                           RUNNING   pid 4904, uptime 0:00:33
thermo_sim                       RUNNING   pid 4905, uptime 0:00:33
trigger_with_pc                  RUNNING   pid 4906, uptime 0:00:33


In [2]:
%run scripts/beamline_configuration.py

## Scalar data to CSV

In [3]:
RE(scan([ph], motor_ph, -1, 1, 5), sample_comp='Ni', barcode='12345')



Transient Scan ID: 3     Time: 2020-05-19 16:31:18
Persistent Unique Scan ID: '830fbc04-1182-4b29-9b03-0a8bcd2d7e49'
New stream: 'primary'


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

+-----------+------------+------------+------------+
|   seq_num |       time |   motor_ph |     ph_det |
+-----------+------------+------------+------------+
|         1 | 16:31:19.0 |     -1.000 |      93483 |


|         2 | 16:31:19.1 |     -0.500 |      94312 |
|         3 | 16:31:19.1 |      0.000 |      94863 |
|         4 | 16:31:19.1 |      0.500 |      94399 |
|         5 | 16:31:19.1 |      1.000 |      93048 |


+-----------+------------+------------+------------+
generator scan ['830fbc04'] (scan num: 3)





('830fbc04-1182-4b29-9b03-0a8bcd2d7e49',)

In [4]:
import suitcase.csv

In [5]:
suitcase.csv.export(db[-1].documents(fill=True), 'my_exported_files/')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/830fbc04-1182-4b29-9b03-0a8bcd2d7e49-primary.csv')]}

In [6]:
suitcase.csv.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}-{start[barcode]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni-12345-primary.csv')]}

In [7]:
suitcase.csv.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345-primary.csv')]}

## Image data to TIFF

In [8]:
RE(scan([spot], mtr_spotx, -1, 1, 5), sample_comp='Ni', barcode='12345')



Transient Scan ID: 4     Time: 2020-05-19 16:31:19
Persistent Unique Scan ID: '286fd335-f356-4bee-a0f8-12da51f29a33'
New stream: 'primary'


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

+-----------+------------+-------------+------------+
|   seq_num |       time | motor_spotx |   spot_roi |
+-----------+------------+-------------+------------+
|         1 | 16:31:19.6 |          -1 | 1057754350 |


|         2 | 16:31:19.7 |          -0 | 1061204919 |


|         3 | 16:31:19.8 |           0 | 1070505372 |


|         4 | 16:31:19.9 |           0 | 1076792320 |


|         5 | 16:31:20.0 |           1 | 1082560588 |


+-----------+------------+-------------+------------+
generator scan ['286fd335'] (scan num: 4)





('286fd335-f356-4bee-a0f8-12da51f29a33',)

### TIFF stack, a 3D block of images in one file

In [9]:
import suitcase.tiff_stack

In [10]:
suitcase.tiff_stack.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345-primary-spot_img.tiff')]}

### TIFF series

In [11]:
import suitcase.tiff_series

Unlike CSV or TIFF stack, this exporter creates separate file per row (Event document). We can include information from a specific Event in the filename. Here, we include the motor position.

In [12]:
suitcase.tiff_series.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}/{event[data][motor_spotx]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/-1.0-primary-spot_img-0.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/-0.5-primary-spot_img-1.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/0.0-primary-spot_img-2.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/0.5-primary-spot_img-3.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/1.0-primary-spot_img-4.tiff')]}

## "Beta" feature: Set up live exporting

*We plan to streamline this in the future, but here is a demonstration that it is possible.*

In [13]:
# Execute this once to configure RE to send data to an exporter.
# At the beamline, this would normally be done in a script automatically at startup.

from event_model import RunRouter
from databroker.core import discover_handlers

def factory(name, start_doc):

    serializer = suitcase.tiff_series.Serializer(
        'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}-{start[uid]:.8}-')

    def cb(name, doc):
        serializer(name, doc)

    return [cb], []


rr = RunRouter([factory], discover_handlers())
RE.subscribe(rr)

2

In [14]:
RE(count([spot], num=30, delay=1), sample_comp='Lab6', barcode='12345')



Transient Scan ID: 5     Time: 2020-05-19 16:31:20
Persistent Unique Scan ID: '45883f0e-0058-4163-8c1c-8f3964cec519'
New stream: 'primary'


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

+-----------+------------+------------+
|   seq_num |       time |   spot_roi |
+-----------+------------+------------+
|         1 | 16:31:20.6 | 1112826550 |


|         2 | 16:31:21.5 | 1128653076 |


|         3 | 16:31:22.5 | 1073238450 |


...

|        28 | 16:31:47.5 | 1059693889 |


|        29 | 16:31:48.5 | 1114305326 |


|        30 | 16:31:49.5 | 1125049525 |


+-----------+------------+------------+
generator count ['45883f0e'] (scan num: 5)





('45883f0e-0058-4163-8c1c-8f3964cec519',)